## 👥 Autores

Este proyecto fue desarrollado por:

| Nombre Completo                     | Número de Identificación | Carrera Universitaria              |
| ----------------------------------- | -----------------------: | ---------------------------------- |
| **Yorladys Argumedo Lozano**        | `1038824209`            | Ingeniería Industrial Virtual      |
| **Sebastian Gabriel Castro**        | `1029720632`            | Ingeniería Industrial Virtual      |

---


In [7]:
# Importamos la librería de Google Colab para manejar archivos
from google.colab import files

# Instalamos la librería de Kaggle
!pip install kaggle --quiet

print("Por favor, sube el archivo 'kaggle.json' que descargaste de Kaggle:")
# Este comando abrirá una ventana para que selecciones el archivo desde tu PC
uploaded = files.upload()

Por favor, sube el archivo 'kaggle.json' que descargaste de Kaggle:


Saving kaggle.json to kaggle.json


In [8]:
# 1. Creamos la carpeta donde Kaggle espera encontrar las credenciales
!mkdir -p ~/.kaggle

# 2. Copiamos el archivo .json a esa carpeta
!cp kaggle.json ~/.kaggle/

# 3. Le damos los permisos de seguridad correctos al archivo
!chmod 600 ~/.kaggle/kaggle.json

print("¡API de Kaggle configurada!")

# 4. AHORA SÍ: Usamos la API para descargar los datos de la competencia
# Este comando le ordena a Colab que descargue los archivos por ti
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

print("\n¡Datos de la competencia descargados!")

# 5. Descomprimimos el archivo .zip que se acaba de descargar
!unzip -q -o udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

print("¡Archivos descomprimidos y listos para usar!")

¡API de Kaggle configurada!
  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.31GB/s]

¡Datos de la competencia descargados!
¡Archivos descomprimidos y listos para usar!


In [18]:
# ==============================================================================
# 1. IMPORTACIÓN DE LIBRERÍAS
# ==============================================================================
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

print("Librerías importadas correctamente.")

# ==============================================================================
# 2. CARGA DEL CONJUNTO DE DATOS
# ==============================================================================
# Usamos la ruta de acceso directo que proporcionaste
ruta_archivo = '/content/train.csv'
train_df = pd.read_csv(ruta_archivo)

print("Archivo train.csv cargado exitosamente.")
train_df.head()

Librerías importadas correctamente.
Archivo train.csv cargado exitosamente.


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [19]:
# ==============================================================================
# 3. PREPROCESADO DE LA VARIABLE OBJETIVO
# ==============================================================================
# La columna 'RENDIMIENTO_GLOBAL' es categórica y necesitamos convertirla a números.

# Creamos un objeto LabelEncoder
le = LabelEncoder()

# Ajustamos y transformamos la columna para crear una nueva columna numérica
train_df['RENDIMIENTO_GLOBAL_encoded'] = le.fit_transform(train_df['RENDIMIENTO_GLOBAL'])

# Mostramos la correspondencia entre categorías y números
print("Mapeo de la variable objetivo:")
for i, clase in enumerate(le.classes_):
    print(f"'{clase}' -> {i}")

# Verificamos los primeros registros de la nueva columna
train_df[['RENDIMIENTO_GLOBAL', 'RENDIMIENTO_GLOBAL_encoded']].head()

Mapeo de la variable objetivo:
'alto' -> 0
'bajo' -> 1
'medio-alto' -> 2
'medio-bajo' -> 3


,RENDIMIENTO_GLOBAL,RENDIMIENTO_GLOBAL_encoded
0,medio-alto,2
1,bajo,1
2,bajo,1
3,alto,0
4,medio-bajo,3


In [20]:
# ==============================================================================
# 4. PREPROCESADO DE VARIABLES CATEGÓRICAS
# ==============================================================================
# Convertiremos las columnas de texto en columnas numéricas que el modelo pueda entender.

# Identificamos las columnas categóricas que vamos a transformar
columnas_categoricas = ['F_ESTRATOVIVIENDA', 'F_TIENEINTERNET']

# Usamos pd.get_dummies para aplicar one-hot encoding
# drop_first=True elimina una de las categorías para evitar redundancia (ej. si no es 'Si', es 'No')
train_df_procesado = pd.get_dummies(train_df, columns=columnas_categoricas, drop_first=True)

print("Columnas después del One-Hot Encoding:")
print(train_df_procesado.columns)

train_df_procesado.head()

Columnas después del One-Hot Encoding:
Index(['ID', 'PERIODO_ACADEMICO', 'E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO',
       'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
       'F_EDUCACIONPADRE', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL',
       'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR',
       'F_TIENEINTERNET.1', 'F_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL',
       'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4',
       'RENDIMIENTO_GLOBAL_encoded', 'F_ESTRATOVIVIENDA_Estrato 2',
       'F_ESTRATOVIVIENDA_Estrato 3', 'F_ESTRATOVIVIENDA_Estrato 4',
       'F_ESTRATOVIVIENDA_Estrato 5', 'F_ESTRATOVIVIENDA_Estrato 6',
       'F_ESTRATOVIVIENDA_Sin Estrato', 'F_TIENEINTERNET_Si'],
      dtype='object')


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,...,INDICADOR_3,INDICADOR_4,RENDIMIENTO_GLOBAL_encoded,F_ESTRATOVIVIENDA_Estrato 2,F_ESTRATOVIVIENDA_Estrato 3,F_ESTRATOVIVIENDA_Estrato 4,F_ESTRATOVIVIENDA_Estrato 5,F_ESTRATOVIVIENDA_Estrato 6,F_ESTRATOVIVIENDA_Sin Estrato,F_TIENEINTERNET_Si
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Técnica o tecnológica incompleta,Si,Si,N,...,0.310,0.267,2,False,True,False,False,False,False,True
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Técnica o tecnológica completa,Si,No,N,...,0.292,0.264,1,False,True,False,False,False,False,False
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Secundaria (Bachillerato) completa,Si,No,N,...,0.305,0.264,1,False,True,False,False,False,False,True
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,No sabe,Si,No,N,...,0.252,0.190,0,False,False,True,False,False,False,True
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Primaria completa,Si,Si,N,...,0.285,0.294,3,False,True,False,False,False,False,True


In [21]:
# ==============================================================================
# 5. MANEJO DE VALORES FALTANTES (IMPUTACIÓN)
# ==============================================================================
# Primero, identifiquemos si hay valores nulos en nuestras columnas numéricas de interés
print("Valores nulos antes de la imputación:")
print(train_df_procesado[['INDICADOR_1']].isnull().sum())

# Si hay valores nulos, los imputaremos con la mediana (que es más robusta a outliers)
# Seleccionamos solo las columnas numéricas para la imputación
columnas_numericas = train_df_procesado.select_dtypes(include=np.number).columns

# Creamos un imputador que reemplace NaN con la mediana
imputador = SimpleImputer(strategy='median')

# Aplicamos la imputación a las columnas numéricas
train_df_procesado[columnas_numericas] = imputador.fit_transform(train_df_procesado[columnas_numericas])

print("\nValores nulos después de la imputación:")
print(train_df_procesado.isnull().sum().sum()) # Debería dar 0 para todo el dataframe si se aplica a todo

Valores nulos antes de la imputación:
INDICADOR_1    0
dtype: int64

Valores nulos después de la imputación:
238612


In [22]:
# ==============================================================================
# 6. ESCALADO DE CARACTERÍSTICAS NUMÉRICAS
# ==============================================================================
# Normalizamos la columna 'INDICADOR_1' para que tenga media 0 y desviación estándar 1.
# Esto es importante para muchos algoritmos de machine learning.

# Creamos un objeto StandardScaler
scaler = StandardScaler()

# Ajustamos y transformamos la columna
train_df_procesado['INDICADOR_1_scaled'] = scaler.fit_transform(train_df_procesado[['INDICADOR_1']])

# Mostramos el antes y el después
print("Estadísticas de 'INDICADOR_1' antes y después del escalado:")
train_df_procesado[['INDICADOR_1', 'INDICADOR_1_scaled']].describe()

Estadísticas de 'INDICADOR_1' antes y después del escalado:


,INDICADOR_1,INDICADOR_1_scaled
count,692500.000000,6.925000e+05
mean,0.268629,-1.178013e-16
std,0.122130,1.000001e+00
min,0.000000,-2.199542e+00
25%,0.203000,-5.373728e-01
50%,0.240000,-2.344159e-01
75%,0.314000,3.714978e-01
max,0.657000,3.179990e+00


In [23]:
# ==============================================================================
# 7. CREACIÓN DEL DATAFRAME FINAL PARA EL MODELO
# ==============================================================================
# Seleccionamos las columnas preprocesadas y la variable objetivo codificada.
# Eliminamos las columnas originales que ya no necesitamos.

# Lista de columnas a eliminar (las originales que fueron codificadas o escaladas)
columnas_a_eliminar = ['RENDIMIENTO_GLOBAL', 'INDICADOR_1']

# Creamos el dataframe final
df_final_modelo = train_df_procesado.drop(columns=columnas_a_eliminar)

# Reordenamos las columnas para tener la variable objetivo al final (es una buena práctica)
objetivo = df_final_modelo.pop('RENDIMIENTO_GLOBAL_encoded')
df_final_modelo['RENDIMIENTO_GLOBAL_encoded'] = objetivo

print("Dimensiones del DataFrame final:", df_final_modelo.shape)
print("\nPrimeras filas del DataFrame listo para el modelo:")
df_final_modelo.head()

Dimensiones del DataFrame final: (692500, 26)

Primeras filas del DataFrame listo para el modelo:


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,...,INDICADOR_4,F_ESTRATOVIVIENDA_Estrato 2,F_ESTRATOVIVIENDA_Estrato 3,F_ESTRATOVIVIENDA_Estrato 4,F_ESTRATOVIVIENDA_Estrato 5,F_ESTRATOVIVIENDA_Estrato 6,F_ESTRATOVIVIENDA_Sin Estrato,F_TIENEINTERNET_Si,INDICADOR_1_scaled,RENDIMIENTO_GLOBAL_encoded
0,904256.0,20212.0,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Técnica o tecnológica incompleta,Si,Si,N,...,0.267,False,True,False,False,False,False,True,0.437002,2.0
1,645256.0,20212.0,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Técnica o tecnológica completa,Si,No,N,...,0.264,False,True,False,False,False,False,False,0.346934,1.0
2,308367.0,20203.0,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Secundaria (Bachillerato) completa,Si,No,N,...,0.264,False,True,False,False,False,False,True,0.232301,1.0
3,470353.0,20195.0,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,No sabe,Si,No,N,...,0.190,False,False,True,False,False,False,True,1.771650,0.0
4,989032.0,20212.0,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Primaria completa,Si,Si,N,...,0.294,False,True,False,False,False,False,True,0.387874,3.0
